From Scracth Implementation 

In [1]:
import numpy as np
import pandas as pd
import os
import re
import string
import random

In [2]:
data_cwe = []
data_sw = []

with open("cwe-test.txt","r") as fileobj:
    for line in fileobj:  
       for i in line: 
           data_cwe+=i

with open("sw-test.txt","r") as fileobj:
    for line in fileobj:  
       for i in line: 
           data_sw+=i

In [3]:
def split_list(a_list):
    half = len(a_list)//2
    return a_list[:half], a_list[half:]

In [4]:
train_cwe, test_cwe = split_list(data_cwe)
train_sw, test_sw = split_list(data_sw)

In [5]:
print("length of training sets", len(train_cwe), len(train_sw))
print("length of test sets", len(test_cwe), len(test_sw))

length of training sets 30858 1725691
length of test sets 30859 1725692


In [6]:
import numpy as np
characters_cwe = np.append(np.unique(data_cwe),"<UNK>")
characters_sw = np.append(np.unique(data_sw),"<UNK>")

In [7]:
print(len(characters_cwe))
print(len(characters_sw))

33
50


In [8]:
def make_markov_model(data, n_gram):
    markov_model = {}
    for i in range(len(data)-n_gram-1):
        curr_state, next_state = "", ""
        for j in range(n_gram):
            curr_state += data[i+j] + ""
            next_state += data[i+j]+ data[j+n_gram] + ""
        curr_state = curr_state[:-1]
        next_state = next_state[:-1]
        if curr_state not in markov_model:
            markov_model[curr_state] = {}
            markov_model[curr_state][next_state] = 1
        else:
            if next_state in markov_model[curr_state]:
                markov_model[curr_state][next_state] += 1
            else:
                markov_model[curr_state][next_state] = 1
    
    # calculating transition probabilities
    for curr_state, transition in markov_model.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_model[curr_state][state] = count/total
        
    return markov_model

In [9]:
train_bigram_model_cwe = make_markov_model(train_cwe,n_gram=2)
print("number of states in bigram model for CWE = ", len(train_bigram_model_cwe.keys()))

train_bigram_model_sw = make_markov_model(train_sw,n_gram=2)
print("number of states in bigram model for SW = ", len(train_bigram_model_sw.keys()))

number of states in bigram model for CWE =  31
number of states in bigram model for SW =  48


In [10]:
train_trigram_model_cwe = make_markov_model(train_cwe,n_gram=3)
print("number of states in trigram model for CWE = ", len(train_trigram_model_cwe.keys()))

train_trigram_model_sw = make_markov_model(train_sw,n_gram=3)
print("number of states in trigram model for SW = ", len(train_trigram_model_sw.keys()))

number of states in trigram model for CWE =  321
number of states in trigram model for SW =  1638


Probability function

In [11]:
def prob(model, word, phrase):
    if phrase in model[word]:
        return model[word][phrase]
    else:
        return 0

In [12]:
print(prob(train_bigram_model_cwe, 'y', 'yno'))
print(prob(train_bigram_model_sw, 'y', 'yia'))
print(prob(train_trigram_model_cwe, 'yu', 'ygu'))
print(prob(train_trigram_model_sw, 'yu', 'y uil'))

0.21888888888888888
0.5784071199067361
0
0.10305343511450382


In [18]:
train_bigram_model_cwe['s']['sna']

0.27358490566037735

Prediction function

In [13]:
def get_prediction(model, phrase):
    return max(model[phrase], key=model[phrase].get)

In [14]:
print(get_prediction(train_bigram_model_cwe, 's'))
print(get_prediction(train_bigram_model_sw, 's'))
print(get_prediction(train_trigram_model_cwe, 'yu'))
print(get_prediction(train_trigram_model_sw, 'yu'))

sna
sih
ygu' 
y uim


Testing data

In [15]:
test_bigram_model_cwe = make_markov_model(train_cwe,n_gram=2)
print("number of states in bigram model for CWE = ", len(test_bigram_model_cwe.keys()))

test_bigram_model_sw = make_markov_model(train_sw,n_gram=2)
print("number of states in bigram model for SW = ", len(test_bigram_model_sw.keys()))

number of states in bigram model for CWE =  31
number of states in bigram model for SW =  48


In [16]:
test_trigram_model_cwe = make_markov_model(train_cwe,n_gram=3)
print("number of states in bigram model for CWE = ", len(test_trigram_model_cwe.keys()))

test_trigram_model_sw = make_markov_model(train_sw,n_gram=3)
print("number of states in bigram model for SW = ", len(test_trigram_model_sw.keys()))

number of states in bigram model for CWE =  321
number of states in bigram model for SW =  1638


Cross entropy

In [40]:
def get_entropy(model,word):
    from math import log2
    entropy = 0
    entropy = entropy -  (1/(len(model[word]))) * log2(model[word][max(model[word], key=model[word].get)])
    return entropy

In [ ]:
for a in test_bigram_model_cwe:
  print("entropy for character ", a , " is ", get_entropy(test_bigram_model_cwe,a))

In [ ]:
for a in test_bigram_model_sw:
  print("entropy for character ", a , " is ", get_entropy(test_bigram_model_sw,a))

In [ ]:
for a in test_trigram_model_cwe:
  print("entropy for character ", a , " is ", get_entropy(test_trigram_model_cwe,a))

In [ ]:
for a in test_trigram_model_sw:
  print("entropy for character ", a , " is ", get_entropy(test_trigram_model_sw,a))